In [ ]:
from dotenv import load_dotenv
import fmpsdk
import pandas as pd
import os

load_dotenv()
apikey = os.environ.get("apikey")
snp500 = pd.DataFrame(fmpsdk.sp500_constituent(apikey))

PBR_KEY = 'priceBookValueRatio'



In [ ]:
from fetch import Fetcher

file_directories = ['financial-ratios-quarter-20220820.csv']
key_list = [PBR_KEY]
key = key_list[0]

fetcher = Fetcher()
fetcher.register(key_list=key_list, csv_file_directory=file_directories[0])
from strategy import *

selector_pbr = Selector(type=SelectorType.HORIZON_RANGE, key=PBR_KEY, value=(0.0, 2.0), horizon=4)
selector_sequence = SelectorSequence(selectors=[selector_pbr])
filter_1 = Filter(selector_sequence_union=[selector_sequence])


In [ ]:
initial_holdings = [Asset(symbol=symbol) for symbol in snp500['symbol']]
start_date = '2017-03-01'
end_date = '2019-03-01'
strategist = Strategist(fetcher=fetcher, asset_pool=initial_holdings, start_date=start_date, end_date=end_date)

strategist.select_assets(filter=filter_1, rebalance_date='2018-03-06')
strategist.select_assets(filter=filter_1, rebalance_date='2018-09-06')


In [4]:
from analyze import Analyst
analyst = Analyst()
analyst.register(table_name='price_list', csv_file_directory='snp500_price_list_19620102_20220814.csv')
analyst.register(table_name='economic_indicators', csv_file_directory='snp500_price_list_19620102_20220814.csv')

In [8]:
import pickle
with open('snp500_price_list_19620102_20220814.pickle', 'rb') as f:
    price_history = pickle.load(f)

In [13]:
df_price_history = pd.DataFrame(price_history)
from pytictoc import TicToc
t = TicToc()
t.tic()
df_price_slice = df_price_history['2001-01-01':'2011-01-01']
t.toc()

Elapsed time is 0.000249 seconds.


In [18]:
df_price_history.index = pd.to_datetime(df_price_history.index)
df_price_history_daily = df_price_history.copy()
df_price_history_monthly = df_price_history.resample('1M').last()
df_price_history_weakly = df_price_history.resample('1w').last()
df_price_history_yearly = df_price_history.resample('1Y').last()